In [1]:
import requests
import pandas as pd
from lxml import etree
import re

In [2]:
def getHtml(url):
    headers = {
        'Cookie': '__atssc=google%3B1; __utma=248915102.1454458327.1560219135.1560219135.1560219135.1; __utmc=248915102; __utmz=248915102.1560219135.1.1.utmcsr=google|utmccn=(organic)|utmcmd=organic|utmctr=(not%20provided); __utmt=1; __atuvc=14%7C24; __atuvs=5cff0dff9219ee5900d; __utmb=248915102.14.10.1560219135',
        'Host': 'www.hok.com',
        'Referer': 'https://www.hok.com/design/service/architecture/adnoc-headquarters/',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.80 Safari/537.36'
    }
    response = requests.get(url, headers = headers)
    response.encoding = response.apparent_encoding
    return response

In [63]:
def getInfos(html):
    
    reg_cont = '//div[@class="column-primary column wysiwyg-primary"]//text()'
    
    info_obj = etree.HTML(html)
    conts = info_obj.xpath(reg_cont)
    if len(conts) == 0:
        conts = None
    
    else:
        conts = '\n'.join(conts)
    
    reg_replace = re.compile('<.*?>|&.*?;|.*?">')
    
    reg_factors = re.compile('<div class="column-small column wysiwyg-lists">(.*?)</div>')
    factors = re.findall(reg_factors, html)
    
    if len(factors) == 0:
        factors = None
    else:
        zeta = factors[0].split('<h5>')[1:]
        factors = ''
        for each in zeta:
            key = re.sub(reg_replace, ' ', each[: each.index('</h5><p>')])
            inner_value = each.split('</h5><p>')[1]
            inner_value = '\n'.join([jj.strip('<p>') for jj in inner_value.split('</p>')]).replace('<br />', '\n').replace('&ndash;','-').replace('&nbsp;',' ').replace('&amp;','&')
            inner_value = re.sub(reg_replace,'',inner_value)
            inner_value = '\n'.join([kk.strip() for kk in inner_value.split('\n')])
            factors += key + ': ' + inner_value + '\n'
        factors = factors.strip('\n')
    return [factors, conts]

In [65]:
def getLinks(html):
    obj = etree.HTML(html)
    
    reg_location = '//h6[@class="location"]/text()'
    reg_alls = re.compile('>Select a Project</option>(.*?)</select>')
    alls = re.findall(reg_alls, html)[0]
    
    locations = obj.xpath(reg_location)
    items = alls.split('</option><option')
    finalResult = []
    
    for kk, item in enumerate(items):

        item = item.split('</option>')[0]
        index_links = item.index('value=') + len('value="')
        index_value = item.index('">')
        item_title = item[index_value + 2: ]
        
        item_link = 'https://www.hok.com' + item[index_links : index_value]
        
        inner_html = getHtml(item_link).text
        inner_info = getInfos(inner_html)
        
        innerResult = [item_title, locations[kk], item_link]
        innerResult.extend(inner_info)
        
        finalResult.append(innerResult)
    return finalResult

In [64]:
final = getLinks(OrigHtml)
print(final)

[['LaGuardia Airport Terminal B', 'New York, New York, USA', 'https://www.hok.com/design/service/architecture/laguardia-airport-terminal-b/', 'LEED CERTIFICATION: LEED NC-Silver anticipated\nSIZE\n1.3 million sq. ft. / 120,700 sq. m.\nSERVICES\nArchitecture\nInterior Design\nMaster Planning\nStructural Engineering\nSustainable Design', 'LaGuardia Airport’s new Terminal B will provide a state-of-the-art passenger terminal and modern New York travel experience.\xa0\nThe 35-gate terminal will replace the existing 1964 Central Terminal Building and optimize use of the airport’s limited airside and landside real estate.\nDesigned to transform LaGuardia Airport into a single, unified hub, the new facility will feature expanded transportation access and best-in-class passenger amenities. A phased construction approach allows it to be built on the constrained site with minimal impact to existing operations.\nTo optimize airfield performance and efficiencies, the team devised an island-concours

In [58]:
result = getLinks(OrigHtml)
print(result)

[['LaGuardia Airport Terminal B', 'New York, New York, USA', 'https://www.hok.com/design/service/architecture/laguardia-airport-terminal-b/', 'LEED CERTIFICATION: LEED NC-Silver anticipated\nspan style="font-size: 11px; font-weight: bold; letter-spacing: 1px; text-transform: uppercase; color: #000000; font-family: helvetica, arial, sans-serif;">SIZE\n1.3 million sq. ft. / 120,700 sq. m.\nspan style="font-size: 11px; font-weight: bold; letter-spacing: 1px; text-transform: uppercase; color: #000000; font-family: helvetica, arial, sans-serif;">SERVICES\nArchitecture\nInterior Design\nMaster Planning\nStructural Engineering\nSustainable Design', 'LaGuardia Airport’s new Terminal B will provide a state-of-the-art passenger terminal and modern New York travel experience.\xa0\nThe 35-gate terminal will replace the existing 1964 Central Terminal Building and optimize use of the airport’s limited airside and landside real estate.\nDesigned to transform LaGuardia Airport into a single, unified h

In [66]:
baseUrl = 'https://www.hok.com/design/service/architecture/'
Original = getHtml(baseUrl)
print(Original)

<Response [200]>


In [67]:
dataSave = pd.DataFrame(result, columns = ['Title','Location','Link','Factors','Content'])
dataSave.to_csv('C:/Users/fred/desktop/HOK.csv',index = False)